Authors: 

Jonathan Naumanen

Adam Williams

Group 17

In [2]:


df_1 = _deepnote_execute_sql("""SELECT *
FROM 'CTG.csv'
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
df_1

,column00,column01,column02,column03,column04,column05,column06,column07,column08,column09,...,column36,column37,column38,column39,column40,column41,column42,column43,column44,column45
0,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,22,None,23
1,b,e,AC,FM,UC,DL,DS,DP,DR,None,...,E,AD,DE,LD,FS,SUSP,None,CLASS,None,NSP
2,240,357,0,0,0,0,0,0,0,None,...,-1,-1,-1,-1,1,-1,None,9,None,2
3,5,632,4,0,4,2,0,0,0,None,...,-1,1,-1,-1,-1,-1,None,6,None,1
4,177,779,2,0,5,2,0,0,0,None,...,-1,1,-1,-1,-1,-1,None,6,None,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2126,1576,3049,1,0,9,0,0,0,0,None,...,1,-1,-1,-1,-1,-1,None,5,None,2
2127,2796,3415,1,1,5,0,0,0,0,None,...,-1,-1,-1,-1,-1,-1,None,1,None,1
2128,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2129,None,None,None,None,None,0,0,0,0,None,...,72,332,252,107,69,197,None,None,None,None


Step 1.

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the CSV file.
data = pd.read_csv("CTG.csv", skiprows=1)
# Select the relevant numerical columns.
selected_cols = ['LB', 'AC', 'FM', 'UC', 'DL', 'DS', 'DP', 'ASTV', 'MSTV', 'ALTV',
                 'MLTV', 'Width', 'Min', 'Max', 'Nmax', 'Nzeros', 'Mode', 'Mean',
                 'Median', 'Variance', 'Tendency', 'NSP']
data = data[selected_cols].dropna()

# Shuffle the dataset.
data_shuffled = data.sample(frac=1.0, random_state=0)

# Split into input part X and output part Y.
X = data_shuffled.drop('NSP', axis=1)

# Map the diagnosis code to a human-readable label.
def to_label(y):
    return [None, 'normal', 'suspect', 'pathologic'][(int(y))]

Y = data_shuffled['NSP'].apply(to_label)

# Partition the data into training and test sets.
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2, random_state=0)

X.head()

,LB,AC,FM,UC,DL,DS,DP,ASTV,MSTV,ALTV,...,Width,Min,Max,Nmax,Nzeros,Mode,Mean,Median,Variance,Tendency
658,130.0,1.0,0.0,3.0,0.0,0.0,0.0,24.0,1.2,12.0,...,35.0,120.0,155.0,1.0,0.0,134.0,133.0,135.0,1.0,0.0
1734,134.0,9.0,1.0,8.0,5.0,0.0,0.0,59.0,1.2,0.0,...,109.0,80.0,189.0,6.0,0.0,150.0,146.0,150.0,33.0,0.0
1226,125.0,1.0,0.0,4.0,0.0,0.0,0.0,43.0,0.7,31.0,...,21.0,120.0,141.0,0.0,0.0,131.0,130.0,132.0,1.0,0.0
1808,143.0,0.0,0.0,1.0,0.0,0.0,0.0,69.0,0.3,6.0,...,27.0,132.0,159.0,1.0,0.0,145.0,144.0,146.0,1.0,0.0
825,152.0,0.0,0.0,4.0,0.0,0.0,0.0,62.0,0.4,59.0,...,25.0,136.0,161.0,0.0,0.0,159.0,156.0,158.0,1.0,1.0


Step 2.

In [4]:
from sklearn.dummy import DummyClassifier

clf = DummyClassifier(strategy='most_frequent')

from sklearn.model_selection import cross_val_score

cvs1 = cross_val_score(clf, Xtrain, Ytrain)
print(cvs1)
sum(cvs1)

[0.78235294 0.78235294 0.77941176 0.77941176 0.77941176]


3.902941176470588

Step 3.

In [5]:
from sklearn.tree import DecisionTreeClassifier

ds_tree_classifier = DecisionTreeClassifier()

cvs2 = cross_val_score(ds_tree_classifier, Xtrain, Ytrain)
print(cvs2)
sum(cvs2)

[0.91764706 0.92647059 0.90882353 0.90882353 0.93823529]


4.6

In [6]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()

cvs3 = cross_val_score(rfc, Xtrain, Ytrain)
print(cvs3)
sum(cvs3)


[0.92941176 0.95294118 0.93823529 0.92941176 0.94411765]


4.694117647058824

In [7]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier()

cvs4 = cross_val_score(gbc, Xtrain, Ytrain)
print(cvs4)
list_of_classifiers = [("", 0)] * 3
list_of_classifiers[0] = ("GBC", sum(cvs4))
sum(cvs4)
print(list_of_classifiers)



[0.94117647 0.96176471 0.94411765 0.94411765 0.95294118]
[('GBC', 4.7441176470588236), ('', 0), ('', 0)]


In [8]:
from sklearn.linear_model import Perceptron

prc = Perceptron()

cvs5 = cross_val_score(prc, Xtrain, Ytrain)
print(cvs5)
sum(cvs5)

[0.84705882 0.88529412 0.82941176 0.85294118 0.71176471]


4.126470588235295

In [9]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter= 100000)

csv6 = cross_val_score(lr, Xtrain, Ytrain)
print(csv6)
list_of_classifiers[1] = ("LR", sum(csv6))
sum(csv6)

[0.88823529 0.9        0.87058824 0.89705882 0.90294118]


4.458823529411765

In [10]:
from sklearn.svm import SVC

svc = SVC()

csv7 = cross_val_score(svc, Xtrain, Ytrain)
print(csv7)
sum(csv7)

[0.85588235 0.87352941 0.83529412 0.85294118 0.87058824]


4.288235294117648

In [11]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier()

csv8 = cross_val_score(mlp, Xtrain, Ytrain)
print(csv8)
list_of_classifiers[2] = ("MLP", sum(csv8))
sum(csv8)

[0.86470588 0.86176471 0.85294118 0.89117647 0.90588235]


4.376470588235295

Step 4. 

In [12]:
from sklearn.metrics import accuracy_score
  
gbc.fit(Xtrain, Ytrain)
Yguess = gbc.predict(Xtest)
print(accuracy_score(Ytest, Yguess))

0.9295774647887324


In [13]:
print(list_of_classifiers)

[('GBC', 4.7441176470588236), ('LR', 4.458823529411765), ('MLP', 4.376470588235295)]


For the report:

Classifier :  MLPClassifier             | GradientBoostingClassifier | LogisticRegression
Accuracy :  4.352941176470588 | 4.741176470588235            | 4.458823529411765

The GradientBoosterClassifier, which is part of the sklearn.ensemble tool-kit gave us the best test results, therefore we used that classifier for the final test. It scored 0.9295774647887324 on the accuracy score.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=190ad4d2-4321-416d-9847-7401b32c451a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>